In [1]:
# timing comparison for gaussian activity computation

In [2]:
from __future__ import print_function

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np 

In [4]:
!gpustat

crescent  Tue Oct  1 16:37:49 2019
[0] GeForce GTX 1080 | 68'C,   0 % |  6812 /  8119 MB | richardso21(151M) salami(6651M)
[1] GeForce GTX 1080 | 67'C,   0 % |  6218 /  8119 MB | richardso21(483M) soumi(1839M) near(2311M) soumi(1573M)


In [5]:
# Define what device we are using
print("CUDA Available: ",torch.cuda.is_available())
use_cuda = False
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")
print("Using device:", device)

# manualy set cuda device

torch.cuda.set_device(1)

CUDA Available:  True
Using device: cpu


In [6]:
batch_size = 10000
out_features = 64
in_features = 28*28
ordinal = 0.5
covar_type = 'diag'

In [7]:
inputs = nn.Parameter(torch.Tensor(batch_size, in_features), requires_grad=True).to(device)
print(inputs.shape[0])

10000


In [8]:
# regular NN weights (transposed at the start, see order of Tensor(dims))
weights = nn.Parameter(torch.Tensor(out_features, in_features), requires_grad=True).to(device)
# centers of FGNs
centers = nn.Parameter(torch.Tensor(out_features, in_features), requires_grad=True).to(device)

In [9]:
#  inverse covariance will actually be used
if covar_type == 'sphere':
    inv_covar = nn.Parameter(torch.Tensor(out_features,), requires_grad=True).to(device)
elif covar_type == 'diag':
    inv_covar = nn.Parameter(torch.Tensor(out_features, in_features,), requires_grad=True).to(device)
elif covar_type == 'full':
    inv_covar = nn.Parameter(torch.Tensor(out_features, in_features, in_features,), requires_grad=True).to(device)

In [10]:
# # method 1: Unsqueeze (large RAM memory reqs)
# # distance to centers
# g = inputs.unsqueeze(1)
# print(g.shape)
# g = g-centers
# print(g.shape)
# g = torch.sum(g, dim=2)
# print(g.shape)
# # g = torch.einsum('zij, ij -> zi', g, inv_covar**2)
# # # g = torch.einsum('zij, ij, zij->zi', g, inv_covar**2, g)
# # print(g.shape)

In [11]:
# print(g)

In [12]:
# # method 2

# # create list params [(in_feats,) for each neuron]
# centers_list = [nn.Parameter(torch.Tensor(in_features,), requires_grad=True) for _ in range(out_features)]
centers_list = [c for c in centers]
inv_covars_list = [i for i in inv_covar]

# # compute distances
# g_list = [torch.sum(inputs-center, dim=1) for center in centers_list]
g_list = [
    torch.sum( 
        torch.pow( 
            (torch.abs(inputs-center)+1e-32)*(inv_covar), ordinal),
        dim=1) 
    for (center,inv_covar) in zip(centers_list, inv_covars_list)
]

print(g_list[0].shape)
g = torch.stack(g_list)
print(g.shape)

g = torch.t(g)
print(g.shape)

# # print(g.shape)
# # for i,x in enumerate(inputs):
# #     g[i] = torch.sum(((x-centers)**2)*inv_covar, dim=1)
    
# # # g = torch.from_numpy(np.array([torch.sum(((x-centers)**2)*inv_covar, dim=1) for x in inputs]))
# # print(g.shape)

KeyboardInterrupt: 

In [ ]:
# print(g)

In [ ]:
%%timeit -n 50
inputs = nn.Parameter(torch.Tensor(batch_size, in_features), requires_grad=True).to(device)
# distance to centers
g = inputs.unsqueeze(1)
g = g-centers
g = torch.sum(g, dim=2)

In [ ]:
%%timeit -n 50
inputs = nn.Parameter(torch.Tensor(batch_size, in_features), requires_grad=True).to(device)
# compute distances
g_list = [torch.sum(inputs-center, dim=1) for center in centers_list]
g = torch.stack(g_list)
g = torch.t(g)

In [5]:
float(False)

0.0